In [72]:
import pandas as pd
import json
import requests
import math

## Design tree
### Patient inputs:
1. Procedure
2. Location
3. Insurance
4. Max distance allowed

### Patient displayed:
1. Sorted list by prices
2. Distances
3. Ratings on a web scraped version


In [168]:
def extract_doctor_card(req):
    result = req["results"][0]
    doctor = {"name": [], "credential": [], "enumeration_date": [], "taxonomy": [], "zipcode": [], "address": [], "address_purpose": [], "phone": []}

    if result["enumeration_type"] == "NPI-1":
        doctor["name"] = result["basic"]["first_name"] + " " + result["basic"]["middle_name"] + " " + result["basic"]["last_name"]
        doctor["credential"] = result["basic"]["credential"]

    else:
        doctor["name"] = result["basic"]["organization_name"]
        doctor["credential"] = "organization"
        
    doctor["enumeration_date"] = result["basic"]["enumeration_date"]

    doctor["taxonomy"] = result["taxonomies"][0]["desc"]
    doctor["zipcode"] = result["addresses"][0]["postal_code"][:5]
    doctor["address"] = result["addresses"][0]["address_1"]
    doctor["address_purpose"] = result["addresses"][0]["address_purpose"]
    doctor["phone"] = result["addresses"][0]["telephone_number"]
    return doctor

def configure_npi_request(npi_id):
    request_str = f"https://npiregistry.cms.hhs.gov/api/?version=2.1&number={npi_id}"
    return request_str

In [120]:
def getHaversineDistance(p1, p2):
    R = 6378137; # Earth’s mean radius in meter
    dLat = rad(p2[0] - p1[0]);
    dLong = rad(p2[1] - p1[1]);
    a = (math.sin(dLat / 2) * math.sin(dLat / 2) + math.cos(rad(p1[0])) * math.cos(rad(p2[0])) * 
         math.sin(dLong / 2) * math.sin(dLong / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c #
    return d # // returns the distance in meter

def meters2miles(meters):
    return meters * 0.000621371

def rad(x):
    return x * math.pi / 180.0;

def get_lat_long(zipcode):
    uri = f'https://public.opendatasoft.com/api/records/1.0/search/?q={zipcode}&dataset=georef-united-states-of-america-zcta5'
    return requests.get(uri).json()["records"][0]["fields"]["geo_point_2d"]

def get_distance(doctor_card, patient_card):
    p_latlong = get_lat_long(patient_card["zipcode"])
    d_latlong = get_lat_long(doctor_card["zipcode"])
    
    d = getHaversineDistance(p_latlong, d_latlong)
    return meters2miles(d)
    

In [130]:
providers = pd.read_csv("../../data/aetna_piecewise/provider_references.csv")
prices = pd.read_csv("../../data/aetna_piecewise/procedure_costs_100000.csv", skiprows=range(1, 10000), nrows=10000)

In [131]:
prices

,Unnamed: 0,negotiation_arrangement,name,billing_code_type,billing_code_type_version,billing_code,description,negotiated_rate,expiration_date,provider_references,negotiated_type,billing_class
0,9999,ffs,NO DOC CUR FUNCT ASSESS,HCPCS,2021.0,G8541,NO DOC CUR FUNCT ASSESS,0.01,9999-12-31,[143549],NaN,institutional
1,10000,ffs,NO DOC CUR FUNCT ASSESS,HCPCS,2021.0,G8541,NO DOC CUR FUNCT ASSESS,0.01,9999-12-31,[219419],NaN,professional
2,10001,ffs,NO DOC CUR FUNCT ASSESS,HCPCS,2021.0,G8541,NO DOC CUR FUNCT ASSESS,0.01,9999-12-31,[471399],NaN,institutional
3,10002,ffs,NO DOC CUR FUNCT ASSESS,HCPCS,2021.0,G8541,NO DOC CUR FUNCT ASSESS,0.01,9999-12-31,[260285],NaN,professional
4,10003,ffs,NO DOC CUR FUNCT ASSESS,HCPCS,2021.0,G8541,NO DOC CUR FUNCT ASSESS,0.01,9999-12-31,[6843],NaN,institutional
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,19994,ffs,ELBOW ARTHROSCOPY,CPT,2021.0,29830,ELBOW ARTHROSCOPY,117.40,9999-12-31,[457371],NaN,institutional
9996,19995,ffs,ELBOW ARTHROSCOPY,CPT,2021.0,29830,ELBOW ARTHROSCOPY,1230.00,9999-12-31,[760939],NaN,institutional
9997,19996,ffs,ELBOW ARTHROSCOPY,CPT,2021.0,29830,ELBOW ARTHROSCOPY,16627.00,9999-12-31,[334802],NaN,institutional
9998,19997,ffs,ELBOW ARTHROSCOPY,CPT,2021.0,29830,ELBOW ARTHROSCOPY,7723.00,9999-12-31,[198302],NaN,institutional


In [132]:
elbow = prices.loc[prices.name.str.contains("ELBOW ARTHROSCOPY", na=False)]

In [134]:
elbow.sort_values("negotiated_rate")

,Unnamed: 0,negotiation_arrangement,name,billing_code_type,billing_code_type_version,billing_code,description,negotiated_rate,expiration_date,provider_references,negotiated_type,billing_class
9857,19856,ffs,ELBOW ARTHROSCOPY,CPT,2021.0,29830,ELBOW ARTHROSCOPY,30.0,9999-12-31,"[305463, 434054, 508373]",NaN,professional
9969,19968,ffs,ELBOW ARTHROSCOPY,CPT,2021.0,29830,ELBOW ARTHROSCOPY,30.0,9999-12-31,[800293],NaN,institutional
9748,19747,ffs,ELBOW ARTHROSCOPY,CPT,2021.0,29830,ELBOW ARTHROSCOPY,30.0,9999-12-31,[394527],NaN,institutional
9952,19951,ffs,ELBOW ARTHROSCOPY,CPT,2021.0,29830,ELBOW ARTHROSCOPY,30.0,9999-12-31,[558051],NaN,professional
9992,19991,ffs,ELBOW ARTHROSCOPY,CPT,2021.0,29830,ELBOW ARTHROSCOPY,30.0,9999-12-31,[506522],NaN,professional
...,...,...,...,...,...,...,...,...,...,...,...,...
9761,19760,ffs,ELBOW ARTHROSCOPY,CPT,2021.0,29830,ELBOW ARTHROSCOPY,15023.0,9999-12-31,"[730488, 790700]",NaN,institutional
9999,19998,ffs,ELBOW ARTHROSCOPY,CPT,2021.0,29830,ELBOW ARTHROSCOPY,15026.0,9999-12-31,[521122],NaN,institutional
9997,19996,ffs,ELBOW ARTHROSCOPY,CPT,2021.0,29830,ELBOW ARTHROSCOPY,16627.0,9999-12-31,[334802],NaN,institutional
9836,19835,ffs,ELBOW ARTHROSCOPY,CPT,2021.0,29830,ELBOW ARTHROSCOPY,21466.0,9999-12-31,[762599],NaN,institutional


In [146]:
references = list(elbow.sort_values("negotiated_rate")["provider_references"])[-1]
print(references)

[130500]


In [140]:
provider_ids = json.loads(references)

In [147]:
npi = list(providers.loc[providers.provider_id.isin(provider_ids)]["npi_list"])[0]
print(npi)

[1043292303, 1285849026]


In [154]:
npi = json.loads(npi)

In [169]:
r = requests.get(configure_npi_request(npi[1]))
doctor = extract_doctor_card(r.json())

In [170]:
doctor

{'name': 'ST. JOSEPH HEALTH NORTHERN CALIFORNIA, LLC',
 'credential': 'organization',
 'enumeration_date': '2007-05-14',
 'taxonomy': 'General Acute Care Hospital, Critical Access',
 'zipcode': '95540',
 'address': '3300 RENNER DR',
 'address_purpose': 'MAILING',
 'phone': '707-725-3361'}

In [116]:
input_card = {"zipcode": 94086, "insurance": "Aetna", "procedure": "ELBOW"}

In [121]:
get_distance(doctor, input_card)

379.7142673439573

In [145]:
doctor

{'name': 'MONICA CECILE NICHOLS',
 'credential': 'MD',
 'enumeration_date': '2006-10-10',
 'gender': 'F',
 'taxonomy': 'Psychiatry & Neurology, Psychiatry',
 'zipcode': '92563',
 'address': '28078 BAXTER RD STE 230',
 'address_purpose': 'LOCATION',
 'phone': '951-824-6116'}